In [1]:
#conda activate torch-xla-nightly
#export XRT_TPU_CONFIG="tpu_worker;0;$10.0.101.2:8470"
#git init
#git remote add origin https://github.com/nosound2/RSNA-Hemorrhage
#git config remote.origin.push HEAD
#git pull origin master
#git config --global branch.master.remote origin
#git config --global branch.master.merge refs/heads/master
#git config remote.origin.push HEAD
#gcloud config set compute/zone europe-west4-a
#gcloud auth login
#gcloud config set project endless-empire-239015
#pip install kaggle --user
#PATH=$PATH:~/.local/bin
#mkdir .kaggle
#gsutil cp gs://recursion-double-strand/kaggle-keys/kaggle.json ~/.kaggle
#chmod 600 /home/zahar_chikishev/.kaggle/kaggle.json
#kaggle competitions download rsna-intracranial-hemorrhage-detection -f stage_1_train.csv
#sudo apt install unzip
#unzip stage_1_train.csv.zip
#kaggle kernels output xhlulu/rsna-generate-metadata-csvs -p .
#kaggle kernels output zaharch/dicom-test-metadata-to-csv -p .
#kaggle kernels output zaharch/dicom-metadata-to-csv -p .
#gsutil cp gs://rsna-hemorrhage/yuvals/* .

#export XRT_TPU_CONFIG="tpu_worker;0;10.0.101.2:8470"; conda activate torch-xla-nightly; jupyter notebook

# 35.233.89.72

# lsblk
# sudo mkfs.ext4 -m 0 -F -E lazy_itable_init=0,lazy_journal_init=0,discard /dev/sdb
# sudo mkdir /mnt/edisk
# sudo mount -o discard,defaults /dev/sdb /mnt/edisk
# sudo chmod a+w /mnt/edisk
# df -H
# echo UUID=`sudo blkid -s UUID -o value /dev/sdb` /mnt/edisk ext4 discard,defaults,nofail 0 2 | sudo tee -a /etc/fstab
# cat /etc/fstab

In [2]:
import sys

from pathlib import Path
from PIL import ImageDraw, ImageFont, Image
from matplotlib import patches, patheffects
import time
from random import randint
import numpy as np
import pandas as pd
import pickle

from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error,log_loss,roc_auc_score
from scipy.stats import ks_2samp

import pdb

import scipy as sp
from tqdm import tqdm, tqdm_notebook

import os
import glob

import torch

#CLOUD = not torch.cuda.is_available()
CLOUD = torch.cuda.get_device_name(0) == 'Tesla K80'
CLOUD_SINGLE = True
TPU = False

if not CLOUD:
    torch.cuda.current_device()

import torch.nn as nn
import torch.utils.data as D
import torch.nn.functional as F
import torch.utils as U

import torchvision
from torchvision import transforms as T
from torchvision import models as M

import matplotlib.pyplot as plt

if CLOUD:
    PATH = Path('/home/zahar_chikishev')
    PATH_WORK = Path('/home/zahar_chikishev/running')
    PATH_DISK = Path('/mnt/edisk/running')
else:
    PATH = Path('C:/StudioProjects/Hemorrhage')
    PATH_WORK = Path('C:/StudioProjects/Hemorrhage/running')
    PATH_DISK = PATH_WORK

from collections import defaultdict, Counter
import random
import seaborn as sn

pd.set_option("display.max_columns", 100)

all_ich = ['any','epidural','intraparenchymal','intraventricular','subarachnoid','subdural']
class_weights = 6.0*np.array([2,1,1,1,1,1])/7.0

if CLOUD and TPU:
    import torch_xla
    import torch_xla.distributed.data_parallel as dp
    import torch_xla.utils as xu
    import torch_xla.core.xla_model as xm

from typing import Collection

In [3]:
all_black = '006d4432e'
all_black = '00bd6c59c'

if CLOUD:
    if TPU:
        device = xm.xla_device()
    else:
        device = 'cuda'
    #device = 'cpu'
    MAX_DEVICES = 1 if CLOUD_SINGLE else 8
    bs = 32
else:
    device = 'cuda'
    #device = 'cpu'
    MAX_DEVICES = 1
    bs = 16

#if CLOUD and (not CLOUD_SINGLE):
#    devices = xm.get_xla_supported_devices(max_devices=MAX_DEVICES)

In [4]:
SEED = 2351

def setSeeds(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

setSeeds(SEED)
torch.backends.cudnn.deterministic = True

In [5]:
def getDSParams(dataset):
    if dataset == 1:
        return 'Densenet161',None,None,2208
    
    elif dataset == 2:
        return 'Densenet169','_3','_v3',208
    elif dataset == 3:
        return 'Densenet201','_3','_v3',240
    elif dataset == 4:
        return 'se_resnext101_32x4d','','',256
    elif dataset == 5:
        return 'Densenet161','_3','_v4',552
    
    elif dataset == 6:
        return 'Densenet201','_3','_v5',240,'',''
    elif dataset == 7:
        return 'Densenet161','_3','_v4',552,'2',''
    elif dataset == 8:
        return 'Densenet169','_3','_v3',208,'2',''
    elif dataset == 9:
        return 'se_resnext101_32x4d','','',256,'','_tta'
    else: assert False

In [6]:
cols_le,cols_float,cols_bool = pickle.load(open(PATH_WORK/'covs','rb'))
meta_cols = cols_bool + cols_float
#meta_cols = [c for c in meta_cols if c not in ['SeriesPP']]
#cols_le = cols_le[:-1]

# Pre-processing

In [7]:
if False:
    if False:
        filename = PATH_WORK/'indexes_file.pkl'
        all_idx, train_ids, val_ids = pickle.load(open(filename,'rb'))

        train_md = pd.read_csv(PATH_WORK/'train_md.csv').sort_values(['SeriesInstanceUID','pos_idx1'])
        train_md['img_id'] = train_md.SOPInstanceUID.str.split('_').apply(lambda x: x[1])

        ids_df = pd.DataFrame(all_idx, columns = ['img_id'])
        ids_df = ids_df.join(train_md.set_index('img_id'), on = 'img_id')
        
        assert len(ids_df.SeriesInstanceUID.unique()) == 19530
        
        trn_data = ids_df.loc[ids_df.img_id.isin(all_idx[train_ids])].reset_index(drop=True)
        val_data = ids_df.loc[ids_df.img_id.isin(all_idx[val_ids])].reset_index(drop=True)

        assert len(trn_data.SeriesInstanceUID.unique()) + len(val_data.SeriesInstanceUID.unique()) \
            == len(train_md.SeriesInstanceUID.unique())

        assert len(trn_data.PatientID.unique()) + len(val_data.PatientID.unique()) \
            >= len(train_md.PatientID.unique())

        pickle.dump((trn_data,val_data), open(PATH_WORK/'train.post.processed.1','wb'))
    else:
        trn_data,val_data = pickle.load(open(PATH_WORK/'train.post.processed.1','rb'))

In [8]:
if False:
    if False:
        test_md = pd.read_csv(PATH_WORK/'test_md.csv').sort_values(['SeriesInstanceUID','pos_idx1'])
        test_md['img_id'] = test_md.SOPInstanceUID.str.split('_').apply(lambda x: x[1])

        filename = PATH_WORK/'test_indexes.pkl'
        test_ids = pickle.load(open(filename,'rb'))

        test_ids_df = pd.DataFrame(test_ids, columns = ['img_id'])
        test_md = test_ids_df.join(test_md.set_index('img_id'), on = 'img_id')

        assert len(test_md.SeriesInstanceUID.unique()) == 2214

        pickle.dump(test_md, open(PATH_WORK/'test.post.processed.1','wb'))
    else:
        test_md = pickle.load(open(PATH_WORK/'test.post.processed.1','rb'))

In [9]:
if False:
    print(pd.concat([test_md[meta_cols].mean(0),
                     trn_data[meta_cols].mean(0),
                     val_data[meta_cols].mean(0)], axis=1))

In [10]:
def loadMetadata(rerun=False):
    if rerun:
        train_dedup = pd.read_csv(PATH_WORK/'yuval'/'train_dedup.csv')
        pids, folding = pickle.load(open(PATH_WORK/'yuval'/'PID_splits.pkl','rb'))

        assert len(pids) == 17079
        assert len(np.unique(pids)) == 17079

        for fol in folding:
            assert len(fol[0]) + len(fol[1]) == 17079

        assert len(folding[0][1]) + len(folding[1][1]) + len(folding[2][1]) == 17079

        assert len(train_dedup.PID.unique()) == 17079

        train_dedup['fold'] = np.nan

        for fold in range(3):
            train_dedup.loc[train_dedup.PID.isin(pids[folding[fold][1]]),'fold'] = fold

        assert train_dedup.fold.isnull().sum() == 0

        train_md = pd.read_csv(PATH_WORK/'train_md.csv').sort_values(['SeriesInstanceUID','pos_idx1'])
        train_md['img_id'] = train_md.SOPInstanceUID.str.split('_').apply(lambda x: x[1])

        ids_df = train_dedup[['fold','PatientID']]
        ids_df.columns = ['fold','img_id']

        train_md = ids_df.join(train_md.set_index('img_id'), on = 'img_id')

        pickle.dump(train_md, open(PATH_WORK/'train.ids.df','wb'))

        #test_md = pickle.load(open(PATH_WORK/'test.post.processed.1','rb'))
        test_md = pd.read_csv(PATH_WORK/'test_md.csv').sort_values(['SeriesInstanceUID','pos_idx1'])
        test_md['img_id'] = test_md.SOPInstanceUID.str.split('_').apply(lambda x: x[1])

        filename = PATH_WORK/'test_indexes.pkl'
        test_ids = pickle.load(open(filename,'rb'))

        test_ids_df = pd.DataFrame(test_ids, columns = ['img_id'])
        test_md = test_ids_df.join(test_md.set_index('img_id'), on = 'img_id')

        assert len(test_md.SeriesInstanceUID.unique()) == 2214

        pickle.dump(test_md, open(PATH_WORK/'test.ids.df','wb'))
    else:
        train_md = pickle.load(open(PATH_WORK/'train.ids.df','rb'))
        test_md = pickle.load(open(PATH_WORK/'test.ids.df','rb'))
    
    return train_md, test_md

In [14]:
#_,_ = loadMetadata(True)

In [18]:
def preprocessedData(dataset, folds = range(3)):
    assert dataset >= 2
    
    dataset_name, filename_add, filename_add2, feat_sz, ds_num, test_fix = getDSParams(dataset)
    
    if 'train_md' not in globals() or 'test_md' not in globals():
        train_md, test_md = loadMetadata(True)
    
    PATH_DS = PATH_DISK/'features/{}{}'.format(dataset_name,filename_add2)
    if not PATH_DS.is_dir():
        PATH_DS.mkdir()
    if not (PATH_DS/'train').is_dir():
        (PATH_DS/'train').mkdir()
    if not (PATH_DS/'test').is_dir():
        (PATH_DS/'test').mkdir()
    
    for fold in folds:
        filename = PATH_DISK/'yuval'/\
            'model_{}{}_version_classifier_splits_type_features_train_tta{}_split_{}.pkl'\
            .format(dataset_name, filename_add, ds_num, fold)
        feats = pickle.load(open(filename,'rb'))
        print('dataset',dataset,'fold',fold,'feats size', feats.shape)
        assert len(feats) == 4*len(train_md)
        assert feats.shape[1] == feat_sz
        means = feats.mean(0,keepdim=True)
        stds = feats.std(0,keepdim=True)

        feats = feats - means
        feats = torch.where(stds > 0, feats/stds, feats)

        for i in range(4):
            feats_sub1 = feats[torch.BoolTensor(np.arange(len(feats))%4 == i)]
            feats_sub2 = feats_sub1[torch.BoolTensor(train_md.fold != fold)]
            pickle.dump(feats_sub2, open(PATH_DS/'train/train.f{}.a{}'.format(fold,i),'wb'))

            feats_sub2 = feats_sub1[torch.BoolTensor(train_md.fold == fold)]
            pickle.dump(feats_sub2, open(PATH_DS/'train/valid.f{}.a{}'.format(fold,i),'wb'))

            if i==0:
                black_feats = feats_sub1[torch.BoolTensor(train_md.img_id == all_black)].squeeze()
                pickle.dump(black_feats, open(PATH_DS/'train/black.f{}'.format(fold),'wb'))

        filename = PATH_DISK/'yuval'/\
            'model_{}{}_version_classifier_splits_type_features_test{}{}_split_{}.pkl'\
            .format(dataset_name,filename_add,ds_num,test_fix,fold)
        #filename = PATH_WORK/'yuval'/\
        #    'model_{}{}_version_classifier_splits_type_features_test_tta_split_{}.pkl'\
        #    .format(dataset_name,filename_add,fold)
        feats = pickle.load(open(filename,'rb'))

        assert len(feats) == 8*len(test_md)
        assert feats.shape[1] == feat_sz
        
        feats = feats - means
        feats = torch.where(stds > 0, feats/stds, feats)

        for i in range(8):
            feats_sub = feats[torch.BoolTensor(np.arange(len(feats))%8 == i)]
            pickle.dump(feats_sub, open(PATH_DS/'test/test.f{}.a{}'.format(fold,i),'wb'))
            assert len(feats_sub) == len(test_md)

In [21]:
#for ds in range(6,10):
#    preprocessedData(ds)

In [85]:
if False:
    if False:
        filename = PATH_WORK/'model_Densenet161_3_vehrsion_basic_classifier_type_features_train_split_2.pkl'
        feats = pickle.load(open(filename,'rb'))

        for series_id in tqdm(ids_df.SeriesInstanceUID.unique()):
            mask = torch.BoolTensor(ids_df.SeriesInstanceUID.values == series_id)
            feats_id = feats[mask]
            pickle.dump(feats_id, open(PATH_WORK/'features/densenet161_v3/train/{}'.format(series_id),'wb'))


        filename = PATH_WORK/'model_Densenet161_3_vehrsion_basic_classifier_type_features_test_split_2.pkl'
        feats = pickle.load(open(filename,'rb'))

        for series_id in tqdm(test_md.SeriesInstanceUID.unique()):
            mask = torch.BoolTensor(test_md.SeriesInstanceUID.values == series_id)
            feats_id = feats[mask]
            pickle.dump(feats_id, open(PATH_WORK/'features/densenet161_v3/test/{}'.format(series_id),'wb'))

# Dataset

In [86]:
if False:
    path = PATH_WORK/'features/densenet161_v3/train/ID_992b567eb6'
    black_feats = pickle.load(open(path,'rb'))[41]

In [87]:
class RSNA_DataSet(D.Dataset):
    def __init__(self, metadata, dataset, mode='train', bs=None, fold=0):
        
        super(RSNA_DataSet, self).__init__()
        
        dataset_name, filename_add, filename_add2, feat_sz,_,_ = getDSParams(dataset)
        self.dataset_name = dataset_name
        self.filename_add2 = filename_add2
        
        if dataset == 1:
            md = metadata.copy()
            md = md.reset_index(drop=True)
        else:
            if mode == 'train':
                md = metadata.loc[metadata.fold != fold].copy().reset_index(drop=True)
            elif mode == 'valid':
                md = metadata.loc[metadata.fold == fold].copy().reset_index(drop=True)
            else:
                md = metadata.copy().reset_index(drop=True)
        
        series = np.sort(md.SeriesInstanceUID.unique())
        md = md.set_index('SeriesInstanceUID', drop=True)
        
        samples_add = 0
        if (mode != 'train') and not DATA_SMALL:
            batch_num = -((-len(series))//(bs*MAX_DEVICES))
            samples_add = batch_num*bs*MAX_DEVICES - len(series)
            print('adding dummy serieses', samples_add)
        
        #self.records = df.to_records(index=False)
        self.mode = mode
        self.real = np.concatenate([np.repeat(True,len(series)),np.repeat(False,samples_add)])
        self.series = np.concatenate([series, random.sample(list(series),samples_add)])
        self.metadata = md
        self.dataset = dataset
        self.fold = fold
        
        print('DataSet', dataset, mode, 'size', len(self.series), 'fold', fold)
        
        if self.dataset >= 2:
            path = PATH_DISK/'features/{}{}/train/black.f{}'.format(self.dataset_name, self.filename_add2, fold)
            self.black_feats = pickle.load(open(path,'rb')).squeeze()
        
        elif self.dataset == 1:
            self.black_feats = black_feats
        
        if WEIGHTED and (self.mode == 'train'):
            tt = self.metadata['weights'].groupby(self.metadata.index).mean()
            self.weights = tt.loc[self.series].values
    
    def setFeats(self, anum, epoch=0):
        def getAPath(an):
            return PATH_DISK/'features/{}{}/{}/{}.f{}.a{}'\
                .format(self.dataset_name,self.filename_add2,folder,self.mode,self.fold,an)
        
        if self.dataset == 1: return
        print('setFeats, augmentation', anum)
        self.anum = anum
        folder = 'test' if self.mode == 'test' else 'train'
        sz = len(self.metadata)
        if anum == -1:
            max_a = 8 if self.mode == 'test' else 4
            feats2 = torch.stack([pickle.load(open(getAPath(an),'rb')) for an in range(max_a)])
            feats = feats2[torch.randint(max_a,(sz,)), torch.arange(sz)].squeeze()
        elif anum == -2:
            max_a = 8 if self.mode == 'test' else 4
            feats2 = torch.stack([pickle.load(open(getAPath(an),'rb')) for an in range(max_a)])
            feats = feats2.mean(0)
        elif anum == -3:
            max_a = 8 if self.mode == 'test' else 4
            feats2 = torch.stack([pickle.load(open(getAPath(an),'rb')) for an in range(max_a)])
            indices_rand = torch.randint(max_a,(sz,))
            indices_rand2 = (torch.randint(1,max_a,(sz,)) + indices_rand) % max_a
            feats = 0.5*(feats2[indices_rand, torch.arange(sz)] + feats2[indices_rand2, torch.arange(sz)]).squeeze()
        else:
            feats = pickle.load(open(getAPath(anum),'rb'))
        
        self.feats = feats
        self.epoch = epoch
        assert len(feats) == sz
    
    def __getitem__(self, index):
        
        series_id = self.series[index]
        #df = self.metadata.loc[self.metadata.SeriesInstanceUID == series_id].reset_index(drop=True)
        df = self.metadata.loc[series_id].reset_index(drop=True)
        
        if self.dataset == 1:
            folder = 'test' if self.mode == 'test' else 'train'
            path = PATH_WORK/'features/densenet161_v3/{}/{}'.format(folder,series_id)
            feats = pickle.load(open(path,'rb'))
            
            if feats.shape[0] > len(df.img_id.unique()):
                mask_dup = ~df.img_id.duplicated().values
                df = df.loc[mask_dup]
                feats = feats[torch.BoolTensor(mask_dup)]
            
            assert feats.shape[0] == len(df)
        elif self.dataset >= 2:
            feats = self.feats[torch.BoolTensor(self.metadata.index.values == series_id)]
        else: assert False
        
        order = np.argsort(df.pos_idx1.values)
        df = df.sort_values(['pos_idx1'])
        feats = feats[torch.LongTensor(order)]
        
        #if WEIGHTED:
        #    non_black = torch.Tensor(df['weights'])
        #else:
        non_black = torch.ones(len(feats))
        
        feats = torch.cat([feats, torch.Tensor(df[meta_cols].values)], dim=1)
        feats_le = torch.LongTensor(df[cols_le].values)
        
        target = torch.Tensor(df[all_ich].values)
        
        #PAD = 4+9+1
        PAD = 12
        
        np.random.seed(1234 + index + 10000*self.epoch)
        offset = np.random.randint(0, 61 - feats.shape[0])
        
        #offset = 0
        bk_add = 0
        top_pad = PAD + offset
        if top_pad > 0:
            dummy_row = torch.cat([self.black_feats, torch.Tensor(df.head(1)[meta_cols].values).squeeze()])
            feats = torch.cat([dummy_row.repeat(top_pad,1), feats], dim=0)
            feats_le = torch.cat([torch.LongTensor(df.head(1)[cols_le].values).squeeze().repeat(top_pad,1), feats_le])
            if offset > 0:
                non_black = torch.cat([bk_add + torch.zeros(offset), non_black])
                target = torch.cat([torch.zeros((offset, len(all_ich))), target], dim=0)
        bot_pad = 60 - len(df) - offset + PAD
        if bot_pad > 0:
            dummy_row = torch.cat([self.black_feats, torch.Tensor(df.tail(1)[meta_cols].values).squeeze()])
            feats = torch.cat([feats, dummy_row.repeat(bot_pad,1)], dim=0)
            feats_le = torch.cat([feats_le, torch.LongTensor(df.tail(1)[cols_le].values).squeeze().repeat(bot_pad,1)])
            if (60 - len(df) - offset) > 0:
                non_black = torch.cat([non_black, bk_add + torch.zeros(60 - len(df) - offset)])
                target = torch.cat([target, torch.zeros((60 - len(df) - offset, len(all_ich)))], dim=0)
        
        assert feats_le.shape[0] == (60 + 2*PAD)
        assert feats.shape[0] == (60 + 2*PAD)
        assert target.shape[0] == 60
        
        idx = index
        if not self.real[index]: idx = -1
        
        if self.mode == 'train':
            return feats, feats_le, target, non_black
        else:
            return feats, feats_le, target, idx, offset
    
    def __len__(self):
        return len(self.series) if not DATA_SMALL else int(0.01*len(self.series))

In [88]:
def getCurrentBatch(dataset, fold=0):
    sel_batch = None
    for filename in os.listdir(PATH_WORK/'models'):
        splits = filename.split('.')
        if int(splits[2][1]) != fold: continue
        if int(splits[3][1:]) != dataset: continue
        if int(splits[4][1:]) != VERSION: continue
        if sel_batch is None:
            sel_batch = int(splits[1][1:])
        else:
            sel_batch = max(sel_batch, int(splits[1][1:]))
    return sel_batch

def modelFileName(dataset, fold=0, batch = 1, return_last = False, return_next = False):
    sel_batch = batch
    if return_last or return_next:
        sel_batch = getCurrentBatch(fold=fold, dataset=dataset)
        if return_last and sel_batch is None:
            return None
        if return_next:
            if sel_batch is None: sel_batch = 1
            else: sel_batch += 1
    
    return 'model.b{}.f{}.d{}.v{}'.format(sel_batch, fold, dataset, VERSION)

# Model

In [3]:
class BCEWithLogitsLoss(nn.Module):
    def __init__(self, weight=None):
        super().__init__()
        self.weight = weight
    
    def forward(self, input, target, batch_weights = None, focal=FOCAL_LOSS):
        if focal == 0:
            loss = (torch.log(1+torch.exp(input)) - target*input)*self.weight
        else:
            loss = torch.pow(1+torch.exp(input), -focal) * \
                (((1-target)*torch.exp(focal*input) + target) * torch.log(1+torch.exp(input)) - target*input)
            loss = loss*self.weight
        if batch_weights is not None:
            loss = batch_weights*loss
        return loss.mean()

NameError: name 'FOCAL_LOSS' is not defined

In [90]:
def bn_drop_lin(n_in:int, n_out:int, bn:bool=True, p:float=0., actn=None):
    "Sequence of batchnorm (if `bn`), dropout (with `p`) and linear (`n_in`,`n_out`) layers followed by `actn`."
    layers = [nn.BatchNorm1d(n_in)] if bn else []
    if p != 0: layers.append(nn.Dropout(p))
    layers.append(nn.Linear(n_in, n_out))
    if actn is not None: layers.append(actn)
    return layers

In [91]:
def noop(x): return x
act_fun = nn.ReLU(inplace=True)

def conv_layer(ni, nf, ks=3, act=True):
    bn = nn.BatchNorm1d(nf)
    layers = [nn.Conv1d(ni, nf, ks), bn]
    if act: layers.append(act_fun)
    return nn.Sequential(*layers)

class ResBlock(nn.Module):
    def __init__(self, ni, nh):
        super().__init__()
        layers  = [conv_layer(ni, nh, 1),
                   conv_layer(nh, nh, 5, act=False)]
        self.convs = nn.Sequential(*layers)
        self.idconv = noop if (ni == nh) else conv_layer(ni, nh, 1, act=False)
    
    def forward(self, x): return act_fun(self.convs(x) + self.idconv(x[:,:,2:-2]))

In [77]:
class ResNetModel(nn.Module):
    def __init__(self, n_cont:int, feat_sz=2208):
        super().__init__()
        
        self.le_sz = 10
        assert self.le_sz == (len(cols_le))
        le_in_sizes = np.array([5,5,7,4,4,11,4,6,3,3])
        le_out_sizes = np.array([3,3,4,2,2,6,2,4,2,2])
        le_out_sz = le_out_sizes.sum()
        self.embeddings = nn.ModuleList([embedding(le_in_sizes[i], le_out_sizes[i]) for i in range(self.le_sz)])
        
        self.feat_sz = feat_sz
        
        self.n_cont = n_cont
        scale = 4
        
        self.conv2D = nn.Conv2d(1,scale*16,(feat_sz + n_cont + le_out_sz,1))
        self.bn1 = nn.BatchNorm1d(scale*16)
        
        self.res1 = ResBlock(scale*16,scale*16)
        self.res2 = ResBlock(scale*16,scale*8)
        
        self.res3 = ResBlock(scale*24,scale*16)
        self.res4 = ResBlock(scale*16,scale*8)
        
        self.res5 = ResBlock(scale*32,scale*16)
        self.res6 = ResBlock(scale*16,scale*8)
        
        self.conv1D = nn.Conv1d(scale*40,6,1)
    
    def forward(self, x, x_le, x_le_mix = None, lambd = None) -> torch.Tensor:
        x_le = [e(x_le[:,:,i]) for i,e in enumerate(self.embeddings)]
        x_le = torch.cat(x_le, 2)
        
        x = torch.cat([x, x_le], 2)
        x = x.transpose(1,2)
        
        #x = torch.cat([x[:,:self.feat_sz],self.bn_cont(x[:,self.feat_sz:])], dim=1)
        x = x.reshape(x.shape[0],1,x.shape[1],x.shape[2])
        
        x = self.conv2D(x).squeeze()
        x = self.bn1(x)
        x = act_fun(x)
        
        x2 = self.res1(x)
        x2 = self.res2(x2)
        
        x3 = torch.cat([x[:,:,4:-4], x2], 1)
        
        x3 = self.res3(x3)
        x3 = self.res4(x3)
        
        x4 = torch.cat([x[:,:,8:-8], x2[:,:,4:-4], x3], 1)
        
        x4 = self.res5(x4)
        x4 = self.res6(x4)
        
        x5 = torch.cat([x[:,:,12:-12], x2[:,:,8:-8], x3[:,:,4:-4], x4], 1)
        x5 = self.conv1D(x5)
        x5 = x5.transpose(1,2)
        
        return x5

In [93]:
def trunc_normal_(x, mean:float=0., std:float=1.):
    "Truncated normal initialization."
    # From https://discuss.pytorch.org/t/implementing-truncated-normal-initializer/4778/12
    return x.normal_().fmod_(2).mul_(std).add_(mean)

In [94]:
def embedding(ni:int,nf:int) -> nn.Module:
    "Create an embedding layer."
    emb = nn.Embedding(ni, nf)
    # See https://arxiv.org/abs/1711.09160
    with torch.no_grad(): trunc_normal_(emb.weight, std=0.01)
    return emb

In [95]:
class TabularModel(nn.Module):
    "Basic model for tabular data."
    def __init__(self, n_cont:int, feat_sz=2208, fc_drop_p=0.3):
        super().__init__()
        self.le_sz = 10
        assert len(cols_le) == self.le_sz
        le_in_sizes = np.array([5,5,7,4,4,11,4,6,3,3])
        le_out_sizes = np.array([3,3,4,2,2,6,2,4,2,2])
        le_out_sz = le_out_sizes.sum()
        self.embeddings = nn.ModuleList([embedding(le_in_sizes[i], le_out_sizes[i]) for i in range(self.le_sz)])
        #self.bn_cont = nn.BatchNorm1d(feat_sz + n_cont)
        
        self.feat_sz = feat_sz
        self.bn_cont = nn.BatchNorm1d(n_cont + le_out_sz)
        self.n_cont = n_cont
        self.fc_drop = nn.Dropout(p=fc_drop_p)
        self.relu = nn.ReLU(inplace=True)
        
        scale = 4
        
        self.conv2D_1 = nn.Conv2d(1,16*scale,(feat_sz + n_cont + le_out_sz,1))
        self.conv2D_2 = nn.Conv2d(1,16*scale,(feat_sz + n_cont + le_out_sz,5))
        self.bn_cont1 = nn.BatchNorm1d(32*scale)
        self.conv1D_1 = nn.Conv1d(32*scale,16*scale,3)
        self.conv1D_3 = nn.Conv1d(32*scale,16*scale,5,dilation=5)
        self.conv1D_2 = nn.Conv1d(32*scale,6,3)
        self.bn_cont2 = nn.BatchNorm1d(32*scale)
        self.bn_cont3 = nn.BatchNorm1d(6)

        self.conv1D_4 = nn.Conv1d(32*scale,32*scale,3)
        self.bn_cont4 = nn.BatchNorm1d(32*scale)

    def forward(self, x, x_le, x_le_mix = None, lambd = None) -> torch.Tensor:
        x_le = [e(x_le[:,:,i]) for i,e in enumerate(self.embeddings)]
        x_le = torch.cat(x_le, 2)
        
        if MIXUP and x_le_mix is not None:
            x_le_mix = [e(x_le_mix[:,:,i]) for i,e in enumerate(self.embeddings)]
            x_le_mix = torch.cat(x_le_mix, 2)
            x_le = lambd * x_le + (1-lambd) * x_le_mix
        
        #assert torch.isnan(x_le).any().cpu() == False
        x = torch.cat([x, x_le], 2)
        x = x.transpose(1,2)
        
        #x = torch.cat([x[:,:self.feat_sz],self.bn_cont(x[:,self.feat_sz:])], dim=1)
        
        x = x.reshape(x.shape[0],1,x.shape[1],x.shape[2])
        x = self.fc_drop(x)
        x = torch.cat([self.conv2D_1(x[:,:,:,2:(-2)]).squeeze(), 
                       self.conv2D_2(x).squeeze()], dim=1)
        x = self.relu(x)
        x = self.bn_cont1(x)
        x = self.fc_drop(x)
        
        x = torch.cat([self.conv1D_1(x[:,:,9:(-9)]),
                       self.conv1D_3(x)], dim=1)
        x = self.relu(x)
        x = self.bn_cont2(x)
        x = self.fc_drop(x)
        
        x = self.conv1D_4(x)
        x = self.relu(x)
        x = self.bn_cont4(x)
        
        x = self.conv1D_2(x)
        x = x.transpose(1,2)
        
        return x

# Training

In [96]:
def train_loop_fn(model, loader, device, context = None):
    
    if CLOUD and (not CLOUD_SINGLE):
        tlen = len(loader._loader._loader)
        OUT_LOSS = 1000
        OUT_TIME = 4
        generator = loader
        device_num = int(str(device)[-1])
        dataset = loader._loader._loader.dataset
    else:
        tlen = len(loader)
        OUT_LOSS = 50
        OUT_TIME = 50
        generator = enumerate(loader)
        device_num = 1
        dataset = loader.dataset
    
    #print('Start training {}'.format(device), 'batches', tlen)
    
    criterion = BCEWithLogitsLoss(weight = torch.Tensor(class_weights).to(device))
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay, betas=(0.9, 0.99))
    
    model.train()
    
    if CLOUD and TPU:
        tracker = xm.RateTracker()

    tloss = 0
    tloss_count = 0
    
    st = time.time()
    mixup_collected = False
    x_le_mix = None
    lambd = None
    for i, (x, x_le, y, non_black) in generator:
        if (not CLOUD) or CLOUD_SINGLE:
            x = x.to(device)
            x_le = x_le.to(device)
            y = y.to(device)
            non_black = non_black.to(device)
        
        if MIXUP:
            if mixup_collected:
                lambd = np.random.beta(0.4, 0.4, y.size(0))
                lambd = torch.Tensor(lambd).to(device)[:,None,None]
                #shuffle = torch.randperm(y.size(0)).to(device)
                x = lambd * x + (1-lambd) * x_mix #x[shuffle]
                #x_le = lambd * x_le + (1-lambd) * x_le_mix #x[shuffle]
                mixup_collected = False
            else:
                x_mix = x
                x_le_mix = x_le
                y_mix = y
                mixup_collected = True
                continue
        
        optimizer.zero_grad()
        output = model(x, x_le, x_le_mix, lambd)
        
        if MIXUP:
            if NO_BLACK_LOSS:
                loss = criterion(output, y, lambd*non_black[:,:,None]) \
                     + criterion(output, y_mix, (1-lambd)*non_black[:,:,None])
            else:
                loss = criterion(output, y, lambd) + criterion(output, y_mix, 1-lambd)
            del x_mix, y_mix
        else:
            if NO_BLACK_LOSS:
                loss = criterion(output, y, non_black[:,:,None])
            else:
                loss = criterion(output, y)
        
        loss.backward()
        
        tloss += len(y)*loss.cpu().detach().item()
        tloss_count += len(y)
        
        if (CLOUD or CLOUD_SINGLE) and TPU:
            xm.optimizer_step(optimizer)
            if CLOUD_SINGLE:
                xm.mark_step()
        else:
            optimizer.step()
        
        if CLOUD and TPU:
            tracker.add(len(y))
        
        st_passed = time.time() - st
        if (i+1)%OUT_TIME == 0 and device_num == 1:
            #print(torch_xla._XLAC._xla_metrics_report())
            print('Batch {} device: {} time passed: {:.3f} time per batch: {:.3f}'
                .format(i+1, device, st_passed, st_passed/(i+1)))
        
        del loss, output, y, x, x_le
    
    return tloss, tloss_count

In [97]:
@torch.no_grad()
def val_loop_fn(model, loader, device, context = None):
    
    if CLOUD and (not CLOUD_SINGLE):
        tlen = len(loader._loader._loader)
        OUT_LOSS = 1000
        OUT_TIME = 4
        generator = loader
        device_num = int(str(device)[-1])
    else:
        tlen = len(loader)
        OUT_LOSS = 1000
        OUT_TIME = 50
        generator = enumerate(loader)
        device_num = 1
    
    #print('Start validating {}'.format(device), 'batches', tlen)
    
    st = time.time()
    model.eval()
    
    results = []
    indices = []
    offsets = []
    
    for i, (x, x_le, y, idx, offset) in generator:
        
        if (not CLOUD) or CLOUD_SINGLE:
            x = x.to(device)
            x_le = x_le.to(device)
        
        output = model(x, x_le)
        assert torch.isnan(output).any().cpu() == False
        output = torch.sigmoid(output)
        assert torch.isnan(output).any().cpu() == False
        
        mask = (idx >= 0)
        results.append(output[mask].cpu().detach().numpy())
        indices.append(idx[mask].cpu().detach().numpy())
        offsets.append(offset[mask].cpu().detach().numpy())
        
        st_passed = time.time() - st
        if (i+1)%OUT_TIME == 0 and device_num == 1:
            print('Batch {} device: {} time passed: {:.3f} time per batch: {:.3f}'
                  .format(i+1, device, st_passed, st_passed/(i+1)))
        
        del output, y, x, x_le, idx, offset
    
    results = np.concatenate(results)
    indices = np.concatenate(indices)
    offsets = np.concatenate(offsets)
    
    return results, indices, offsets

In [98]:
@torch.no_grad()
def test_loop_fn(model, loader, device, context = None):
    
    if CLOUD and (not CLOUD_SINGLE):
        tlen = len(loader._loader._loader)
        OUT_LOSS = 1000
        OUT_TIME = 100
        generator = loader
        device_num = int(str(device)[-1])
    else:
        tlen = len(loader)
        OUT_LOSS = 1000
        OUT_TIME = 10
        generator = enumerate(loader)
        device_num = 1
    
    #print('Start testing {}'.format(device), 'batches', tlen)
    
    st = time.time()
    model.eval()
    
    results = []
    indices = []
    offsets = []
    
    for i, (x, x_le, y, idx, offset) in generator:
        
        if (not CLOUD) or CLOUD_SINGLE:
            x = x.to(device)
            x_le = x_le.to(device)
        
        output = torch.sigmoid(model(x, x_le))
        
        mask = (idx >= 0)
        results.append(output[mask].cpu().detach().numpy())
        indices.append(idx[mask].cpu().detach().numpy())
        offsets.append(offset[mask].cpu().detach().numpy())
        
        st_passed = time.time() - st
        if (i+1)%OUT_TIME == 0 and device_num == 1:
            print('B{} -> time passed: {:.3f} time per batch: {:.3f}'.format(i+1, st_passed, st_passed/(i+1)))
        
        del output, x, y, idx, offset
    
    return np.concatenate(results), np.concatenate(indices), np.concatenate(offsets)

In [76]:
def train_one(dataset, weight=None, load_model=True, epochs=1, bs=100, fold=0):
    
    st0 = time.time()
    dataset_name, filename_add, filename_add2, feat_sz,_,_ = getDSParams(dataset)
    
    cur_epoch = getCurrentBatch(fold=fold, dataset=dataset)
    if cur_epoch is None: cur_epoch = 0
    print('completed epochs:', cur_epoch, 'starting now:', epochs)
    
    setSeeds(SEED + cur_epoch)
    
    if dataset == 1:
        trn_ds = RSNA_DataSet(trn_data, mode='train', bs=bs, fold=fold, dataset=dataset)
        val_ds = RSNA_DataSet(val_data, mode='valid', bs=bs, fold=fold, dataset=dataset)
    elif dataset >= 2:
        trn_ds = RSNA_DataSet(train_md, mode='train', bs=bs, fold=fold, dataset=dataset)
        val_ds = RSNA_DataSet(train_md, mode='valid', bs=bs, fold=fold, dataset=dataset)
    else: assert False
    val_ds.setFeats(0)
    
    if WEIGHTED:
        print('WeightedRandomSampler')
        sampler = D.sampler.WeightedRandomSampler(trn_ds.weights, len(trn_ds))
        loader = D.DataLoader(trn_ds, num_workers=8 if CLOUD else 0, batch_size=bs, 
                              shuffle=False, drop_last=True, sampler=sampler)
    else:
        loader = D.DataLoader(trn_ds, num_workers=8 if CLOUD else 0, batch_size=bs, 
                              shuffle=True, drop_last=True)
    loader_val = D.DataLoader(val_ds, num_workers=8 if CLOUD else 0, batch_size=bs, 
                              shuffle=True)
    print('dataset train:', len(trn_ds), 'valid:', len(val_ds), 'loader train:', len(loader), 'valid:', len(loader_val))
    
    #model = TabularModel(n_cont = len(meta_cols), feat_sz=feat_sz, fc_drop_p=0)
    model = ResNetModel(n_cont = len(meta_cols), feat_sz=feat_sz)
    
    model_file_name = modelFileName(return_last=True, fold=fold, dataset=dataset)
    if model_file_name is not None:
        print('loading model', model_file_name)
        state_dict = torch.load(PATH_WORK/'models'/model_file_name)
        model.load_state_dict(state_dict)
    else:
        print('starting from scratch')
    
    if (not CLOUD) or CLOUD_SINGLE:
        model = model.to(device)
    else:
        model_parallel = dp.DataParallel(model, device_ids=devices)
    
    loc_data = val_ds.metadata.copy()
    
    if DATA_SMALL:
        #val_sz = int(0.01*len(loc_data.index.unique()))
        val_sz = len(val_ds)
        val_series = val_ds.series[:val_sz]
        loc_data = loc_data.loc[loc_data.index.isin(val_series)]
    
    series_counts = loc_data.index.value_counts()
    
    loc_data['orig_idx'] = np.arange(len(loc_data))
    loc_data = loc_data.sort_values(['SeriesInstanceUID','pos_idx1'])
    loc_data['my_order'] = np.arange(len(loc_data))
    loc_data = loc_data.sort_values(['orig_idx'])
    
    ww_val = loc_data.weights
    
    for i in range(cur_epoch+1, cur_epoch+epochs+1):
        st = time.time()
        
        #trn_ds.setFeats((i-1) % 4)
        trn_ds.setFeats(-1, epoch=i)
        
        if CLOUD and (not CLOUD_SINGLE):
            results = model_parallel(train_loop_fn, loader)
            tloss, tloss_count = np.stack(results).sum(0)
            state_dict = model_parallel._models[0].state_dict()
        else:
            tloss, tloss_count = train_loop_fn(model, loader, device)
            state_dict = model.state_dict()
        
        state_dict = {k:v.to('cpu') for k,v in state_dict.items()}
        tr_ll = tloss / tloss_count
        
        train_time = time.time()-st
        
        model_file_name = modelFileName(return_next=True, fold=fold, dataset=dataset)
        if not DATA_SMALL:
            torch.save(state_dict, PATH_WORK/'models'/model_file_name)
        
        st = time.time()
        if CLOUD and (not CLOUD_SINGLE):
            results = model_parallel(val_loop_fn, loader_val)
            predictions = np.concatenate([results[i][0] for i in range(MAX_DEVICES)])
            indices = np.concatenate([results[i][1] for i in range(MAX_DEVICES)])
            offsets = np.concatenate([results[i][2] for i in range(MAX_DEVICES)])
        else:
            predictions, indices, offsets = val_loop_fn(model, loader_val, device)
        
        predictions = predictions[np.argsort(indices)]
        offsets = offsets[np.argsort(indices)]
        assert len(predictions) == len(loc_data.index.unique())
        assert len(predictions) == len(offsets)
        assert np.all(indices[np.argsort(indices)] == np.array(range(len(predictions))))
        
        #val_results = np.zeros((len(loc_data),6))
        val_results = []
        for k, series in enumerate(np.sort(loc_data.index.unique())):
            cnt = series_counts[series]
            #mask = loc_data.SeriesInstanceUID == series
            assert (offsets[k] + cnt) <= 60
            #val_results[mask] = predictions[k,offsets[k]:(offsets[k] + cnt)]
            val_results.append(predictions[k,offsets[k]:(offsets[k] + cnt)])
        
        val_results = np.concatenate(val_results)
        assert np.isnan(val_results).sum() == 0
        val_results = val_results[loc_data.my_order]
        assert np.isnan(val_results).sum() == 0
        assert len(val_results) == len(loc_data)
        
        lls = [log_loss(loc_data[all_ich[k]].values, val_results[:,k], eps=1e-7, labels=[0,1])\
               for k in range(6)]
        ll = (class_weights * np.array(lls)).mean()
        cor = np.corrcoef(loc_data.loc[:,all_ich].values.reshape(-1), val_results.reshape(-1))[0,1]
        auc = roc_auc_score(loc_data.loc[:,all_ich].values.reshape(-1), val_results.reshape(-1))
        
        lls_w = [log_loss(loc_data[all_ich[k]].values, val_results[:,k], eps=1e-7, labels=[0,1], sample_weight=ww_val)\
               for k in range(6)]
        ll_w = (class_weights * np.array(lls_w)).mean()
        
        print('v{}, d{}, e{}, f{}, trn ll: {:.4f}, val ll: {:.4f}, ll_w: {:.4f}, cor: {:.4f}, '
              'auc: {:.4f}, lr: {}'.format(VERSION, dataset, i, fold, tr_ll, ll, ll_w, cor, auc, learning_rate))
        valid_time = time.time()-st
        
        epoch_stats = pd.DataFrame([[dataset, i, fold, tr_ll, ll, ll_w, cor, lls[0], lls[1], lls[2], lls[3], 
                                     lls[4], lls[5], len(trn_ds), len(val_ds), bs, train_time, valid_time,
                                     learning_rate, weight_decay]],
                                   columns = 
                                    ['dataset','epoch','fold','train_loss','val_loss','val_w_loss','cor',
                                     'any','epidural','intraparenchymal','intraventricular','subarachnoid','subdural',
                                     'train_sz','val_sz','bs','train_time','valid_time','lr','wd'
                                     ])

        stats_filename = PATH_WORK/'stats.f{}.v{}'.format(fold,VERSION)
        if stats_filename.is_file():
            epoch_stats = pd.concat([pd.read_csv(stats_filename), epoch_stats], sort=False)
        #if not DATA_SMALL:
        epoch_stats.to_csv(stats_filename, index=False)
    
    print('total running time', time.time() - st0)
    
    return model, predictions, val_results

# OOF

In [100]:
def oof_one(dataset, num_iter=1, bs=100, fold=0):
    
    st0 = time.time()
    dataset_name, filename_add, filename_add2, feat_sz,_,_ = getDSParams(dataset)
    
    cur_epoch = getCurrentBatch(fold=fold, dataset=dataset)
    if cur_epoch is None: cur_epoch = 0
    print('completed epochs:', cur_epoch, 'iters starting now:', num_iter)
    
    setSeeds(SEED + cur_epoch)
    
    val_ds = RSNA_DataSet(train_md, mode='valid', bs=bs, fold=fold, dataset=dataset)
    
    loader_val = D.DataLoader(val_ds, num_workers=8 if CLOUD else 0, batch_size=bs, 
                              shuffle=True)
    print('dataset valid:', len(val_ds), 'loader valid:', len(loader_val))
    
    #model = TabularModel(n_cont = len(meta_cols), feat_sz=feat_sz, fc_drop_p=0)
    model = ResNetModel(n_cont = len(meta_cols), feat_sz=feat_sz)
    
    model_file_name = modelFileName(return_last=True, fold=fold, dataset=dataset)
    if model_file_name is not None:
        print('loading model', model_file_name)
        state_dict = torch.load(PATH_WORK/'models'/model_file_name)
        model.load_state_dict(state_dict)
    else:
        print('starting from scratch')
    
    if (not CLOUD) or CLOUD_SINGLE:
        model = model.to(device)
    else:
        model_parallel = dp.DataParallel(model, device_ids=devices)
    
    loc_data = val_ds.metadata.copy()
    series_counts = loc_data.index.value_counts()
    
    loc_data['orig_idx'] = np.arange(len(loc_data))
    loc_data = loc_data.sort_values(['SeriesInstanceUID','pos_idx1'])
    loc_data['my_order'] = np.arange(len(loc_data))
    loc_data = loc_data.sort_values(['orig_idx'])
    
    preds = []
    
    for i in range(num_iter):
        
        val_ds.setFeats(-1)

        if CLOUD and (not CLOUD_SINGLE):
            results = model_parallel(val_loop_fn, loader_val)
            predictions = np.concatenate([results[i][0] for i in range(MAX_DEVICES)])
            indices = np.concatenate([results[i][1] for i in range(MAX_DEVICES)])
            offsets = np.concatenate([results[i][2] for i in range(MAX_DEVICES)])
        else:
            predictions, indices, offsets = val_loop_fn(model, loader_val, device)

        predictions = predictions[np.argsort(indices)]
        offsets = offsets[np.argsort(indices)]
        assert len(predictions) == len(loc_data.index.unique())
        assert len(predictions) == len(offsets)
        assert np.all(indices[np.argsort(indices)] == np.array(range(len(predictions))))

        val_results = []
        for k, series in enumerate(np.sort(loc_data.index.unique())):
            cnt = series_counts[series]
            assert (offsets[k] + cnt) <= 60
            val_results.append(predictions[k,offsets[k]:(offsets[k] + cnt)])

        val_results = np.concatenate(val_results)
        assert np.isnan(val_results).sum() == 0
        val_results = val_results[loc_data.my_order]
        assert np.isnan(val_results).sum() == 0
        assert len(val_results) == len(loc_data)
        
        preds.append(val_results)

        lls = [log_loss(loc_data[all_ich[k]].values, val_results[:,k], eps=1e-7, labels=[0,1])\
               for k in range(6)]
        ll = (class_weights * np.array(lls)).mean()
        cor = np.corrcoef(loc_data.loc[:,all_ich].values.reshape(-1), val_results.reshape(-1))[0,1]
        auc = roc_auc_score(loc_data.loc[:,all_ich].values.reshape(-1), val_results.reshape(-1))

        print('ver {}, iter {}, fold {}, val ll: {:.4f}, cor: {:.4f}, auc: {:.4f}'
              .format(VERSION, i, fold, ll, cor, auc))
    
    print('total running time', time.time() - st0)
    
    return np.stack(preds)

## Selecting runs aggregation

In [1]:
afuncs_names = np.array(['mean','gmean','q50','q25','q75','psig'])
afuncs_names = np.array(['mean','gmean','psig'])

def scalePreds(x, power = 2, center=0.5):
    res = x.copy()
    res[x > center] = center + (1 - center) * ((res[x > center] - center)/(1 - center))**power
    res[x < center] = center - center * ((center - res[x < center])/center)**power
    return res

def applyAggFunc(probs, func_name, axis=1, norm_axis=None):
    
    st = time.time()
    
    if func_name == 'mean':
        res = probs.mean(axis)
    elif func_name == 'max':
        res = probs.max(axis)
    elif func_name == 'min':
        res = probs.min(axis)
    elif func_name == 'gmean':
        res = np.exp(np.log(probs).mean(axis))
    elif func_name == 'q50':
        res = np.quantile(probs,q=0.5,axis=axis)
    elif func_name == 'q25':
        res = np.quantile(probs,q=0.25,axis=axis)
    elif func_name == 'q75':
        res = np.quantile(probs,q=0.75,axis=axis)
    elif func_name == 'psig':
        mask_zero = (probs >= 1).sum(axis) > 0
        res = 1 / (1 + np.exp(-(np.log(probs/(1-probs), where=(probs < 1)).mean(axis))))
        res[mask_zero] = 0
    else:
        assert True
    
    res = np.clip(res, 1e-15, 1-1e-15)
    
    #print('applying time', func_name, time.time()-st)
    return res

def getPredsOOF(aug=32,datasets=range(6,10),ver=VERSION):
    preds_all = np.zeros((4,aug,len(train_md),6))
    
    for fold in range(3):
        preds = np.stack([pickle.load(open(PATH_DISK/'ensemble/oof_d{}_f{}_v{}'
                                           .format(ds, fold, ver),'rb')) for ds in datasets])
        preds_all[:,:,train_md.fold == fold,:] = preds
    
    preds_all = np.clip(preds_all, 1e-15, 1-1e-15)
    return preds_all

NameError: name 'np' is not defined

In [ ]:
def getYuvalOOF(names, train_md, aug=32):
    
    if 'yuval_idx' not in train_md.columns:
        print('adding yuval_idx')
        yuval_oof = pickle.load(open(PATH_DISK/'ensemble/OOF_validation_image_ids.pkl','rb'))

        df2 = pd.DataFrame()
        for fold in range(3):
            df = pd.DataFrame(np.arange(len(yuval_oof[fold])), columns=['yuval_idx'])
            df.index = yuval_oof[fold]
            df2 = pd.concat([df2,df],sort=False)

        train_md = train_md.join(df2, on = 'img_id')        
    
    preds_y = np.zeros((len(names),aug,len(train_md),6))
    
    for fold in range(3):
        preds = np.stack([torch.sigmoid(torch.stack(pickle.load(open(PATH_DISK/'ensemble'/name.format(fold),'rb'))))
                          .numpy() for name in names])
        preds = preds[:,:,train_md.loc[train_md.fold == fold, 'yuval_idx']]
        
        preds_y[:,:,train_md.fold == fold] = preds
    
    preds_y = np.clip(preds_y, 1e-15, 1-1e-15)[:,:,:,np.array([5,0,1,2,3,4])]
    return preds_y

# Inference

In [102]:
def inference_one(dataset, bs = 100, add_seed = 0, fold = 0, anum = 0):
    
    st = time.time()
    dataset_name, filename_add, filename_add2, feat_sz,_,_ = getDSParams(dataset)

    cur_epoch = getCurrentBatch(fold=fold, dataset=dataset)
    if cur_epoch is None: cur_epoch = 0
    print('completed epochs:', cur_epoch)

    #model = TabularModel(n_cont = len(meta_cols), feat_sz=feat_sz)
    model = ResNetModel(n_cont = len(meta_cols), feat_sz=feat_sz)
    
    model_file_name = modelFileName(return_last=True, fold=fold, dataset=dataset)
    if model_file_name is not None:
        print('loading model', model_file_name)
        state_dict = torch.load(PATH_WORK/'models'/model_file_name)
        model.load_state_dict(state_dict)
    
    if (not CLOUD) or CLOUD_SINGLE:
        model = model.to(device)
    else:
        model_parallel = dp.DataParallel(model, device_ids=devices)

    setSeeds(SEED + cur_epoch + anum + 100*fold)

    tst_ds = RSNA_DataSet(test_md, mode='test', bs=bs, fold=fold, dataset=dataset)
    loader_tst = D.DataLoader(tst_ds, num_workers=8 if CLOUD else 0, batch_size=bs, shuffle=False)
    print('dataset test:', len(tst_ds), 'loader test:', len(loader_tst))
    
    tst_ds.setFeats(-1)

    loc_data = tst_ds.metadata.copy()
    series_counts = loc_data.index.value_counts()

    loc_data['orig_idx'] = np.arange(len(loc_data))
    loc_data = loc_data.sort_values(['SeriesInstanceUID','pos_idx1'])
    loc_data['my_order'] = np.arange(len(loc_data))
    loc_data = loc_data.sort_values(['orig_idx'])
    
    if CLOUD and (not CLOUD_SINGLE):
        results = model_parallel(test_loop_fn, loader_tst)
        predictions = np.concatenate([results[i][0] for i in range(MAX_DEVICES)])
        indices = np.concatenate([results[i][1] for i in range(MAX_DEVICES)])
        offsets = np.concatenate([results[i][2] for i in range(MAX_DEVICES)])
    else:
        predictions, indices, offsets = test_loop_fn(model, loader_tst, device)

    predictions = predictions[np.argsort(indices)]
    offsets = offsets[np.argsort(indices)]
    assert len(predictions) == len(test_md.SeriesInstanceUID.unique())
    assert np.all(indices[np.argsort(indices)] == np.array(range(len(predictions))))
    
    val_results = []
    for k, series in enumerate(np.sort(loc_data.index.unique())):
        cnt = series_counts[series]
        assert (offsets[k] + cnt) <= 60
        val_results.append(predictions[k,offsets[k]:(offsets[k] + cnt)])

    val_results = np.concatenate(val_results)
    assert np.isnan(val_results).sum() == 0
    val_results = val_results[loc_data.my_order]
    assert len(val_results) == len(loc_data)

    print('test processing time:', time.time() - st)
    
    return val_results

# Ensembling

In [1]:
def getSecondStepX(train_md, preds_all, TH, powerLow=0.7, powerHigh=1.5, fold=0, target=0, ds_idx=0, mode='train'):
    
    X = np.stack([pickle.load(open(PATH_DISK/'ensemble'/'first.d{}.f{}.t{}.v{}'
                                   .format(ds_idx,fold,target,VERSION),'rb')) for ds_idx in range(len(preds_all))])
    
    if mode == 'train':
        X = X[:,train_md.fold != fold]
        y = train_md.loc[train_md.fold != fold, all_ich[target]].values
    elif mode == 'valid':
        X = X[:,train_md.fold == fold]
        y = train_md.loc[train_md.fold == fold, all_ich[target]].values
    else:
        X = X
        y = None
    
    ll = None
    auc = None
    if y is not None:
        ll = log_loss(y, X.mean(0), eps=1e-7, labels=[0,1])
        auc = roc_auc_score(y, X.mean(0))
    
    return X, y, ll, auc


def getFirstStepX(train_md, preds_all, TH, powerLow=0.7, powerHigh=1.5, fold=0, target=0, ds_idx=0, mode='train'):
    
    X0 = preds_all[ds_idx]
    X0 = X0.mean(0)
    
    if mode == 'train':
        X0 = X0[train_md.fold != fold]
        y = train_md.loc[train_md.fold != fold, all_ich[target]].values
    elif mode == 'valid':
        X0 = X0[train_md.fold == fold]
        y = train_md.loc[train_md.fold == fold, all_ich[target]].values
    else:
        X0 = X0
        y = None
    
    X0 = X0[:,target]
    
    ll = None
    auc = None
    if y is not None:
        ll = log_loss(y, X0, eps=1e-7, labels=[0,1])
        auc = roc_auc_score(y, X0)
    
    X = np.stack([np.where(X0 > TH,X0,np.zeros(X0.shape)),
                  np.where(X0 > TH,scalePreds(X0,power=powerLow,center=TH),np.zeros(X0.shape)),
                  np.where(X0 > TH,scalePreds(X0,power=powerHigh,center=TH),np.zeros(X0.shape)),
                  np.where(X0 <= TH,X0,np.zeros(X0.shape)),
                  np.where(X0 <= TH,scalePreds(X0,power=powerLow,center=TH),np.zeros(X0.shape)),
                  np.where(X0 <= TH,scalePreds(X0,power=powerHigh,center=TH),np.zeros(X0.shape)),
                  ])
    
    return X, y, ll, auc


def train_ensemble(train_md, preds_all, fold = 0, target = 0, ds_idx = 0, first_step = True, skip_first=False):
    
    if first_step:
        print('starting model',ds_idx,'fold',fold,'target',target)
    else:
        print('starting fold',fold,'target',target)
    
    if skip_first:
        total_preds = preds_all[ds_idx].mean(0)[:,target]
        pickle.dump(total_preds, open(PATH_DISK/'ensemble'/'first.d{}.f{}.t{}.v{}'
                                      .format(ds_idx,fold,target,VERSION),'wb'))
        return
    
    st = time.time()
    intercept = False
    
    limit_low = 1e-15
    limit_high = 1 - 1e-4
    
    prior = train_md.loc[train_md.fold != fold, all_ich[target]].mean()
    powerLow = 0.7
    powerHigh = 1.5
    
    def my_objective(x,preds,vals):
        preds_sum = np.clip((preds * x[:,None]).sum(0), limit_low, limit_high)
        res = (- vals * np.log(preds_sum) - (1 - vals) * np.log(1 - preds_sum)).mean()
        #print('x   ',x, x.sum())
        print('obj ',res)
        return res

    def my_grad(x,preds,vals):
        preds_sum = np.clip((preds * x[:,None]).sum(0), limit_low, limit_high)
        res = (- vals * preds / preds_sum + (1 - vals) * preds / (1 - preds_sum)).mean(1)
        #print('grad',res)
        return res

    def my_hess(x,preds,vals):
        preds_sum = np.clip((preds * x[:,None]).sum(0), limit_low, limit_high)
        res = (preds * np.expand_dims(preds, axis=1) * (vals / preds_sum**2 + (1 - vals) / (1 - preds_sum)**2)).mean(2)
        return res
    
    if first_step: data_func = getFirstStepX
    else: data_func = getSecondStepX
    
    X,y,ll_train,auc_train =  data_func(train_md, preds_all, TH=prior, powerLow=powerLow, powerHigh=powerHigh, 
                                        fold=fold, target=target, ds_idx=ds_idx)
    
    bnds_low = np.zeros(X.shape[0])
    bnds_high = np.ones(X.shape[0])
    
    if first_step:
        initial_sol = np.array([1,0,0,1,0,0])
        splitted = True
    else:
        initial_sol = np.ones(len(preds_all))/len(preds_all)
        splitted = False
    
    if intercept:
        X = np.concatenate([X,np.ones((1,X.shape[1]))], axis=0)
        bnds_low = np.concatenate([bnds_low, -0.1*np.ones(1)])
        bnds_high = np.concatenate([bnds_high, 0.1*np.ones(1)])
        initial_sol = np.concatenate([initial_sol, np.zeros(1)])

    bnds = sp.optimize.Bounds(bnds_low, bnds_high)
    cons = sp.optimize.LinearConstraint(np.ones((1,X.shape[0])), 0.95 + splitted, 1.01 + splitted)
    
    model = sp.optimize.minimize(my_objective, initial_sol, jac=my_grad, hess=my_hess, args=(X, y),
                                 bounds=bnds, method='trust-constr', constraints=cons,
                                 options={'gtol': 1e-11, 'initial_tr_radius': 0.1, 'initial_barrier_parameter': 0.01})
    model.prior = prior
    model.powerLow = powerLow
    model.powerHigh = powerHigh
    
    pickle.dump(model, open(PATH_DISK/'ensemble'/'model.d{}.f{}.t{}.v{}'
                            .format(ds_idx,fold,target,VERSION),'wb'))

    train_preds = (X*np.expand_dims(model.x, axis=1)).sum(0)
    ll_train2 = log_loss(y, train_preds, eps=1e-7, labels=[0,1])
    auc_train2 = roc_auc_score(y, train_preds)
    
    X,y,ll_val,auc_val =  data_func(train_md, preds_all, TH=prior, powerLow=powerLow, powerHigh=powerHigh, 
                                    fold=fold, target=target, ds_idx=ds_idx, mode='valid')
    
    val_preds = (X*np.expand_dims(model.x, axis=1)).sum(0)
    
    if first_step:
        total_preds = np.zeros(len(train_md))
        total_preds[train_md.fold == fold] = val_preds
        total_preds[train_md.fold != fold] = train_preds
        pickle.dump(total_preds, open(PATH_DISK/'ensemble'/'first.d{}.f{}.t{}.v{}'
                                      .format(ds_idx,fold,target,VERSION),'wb'))
    
    ll_val2 = log_loss(y, val_preds, eps=1e-7, labels=[0,1])
    auc_val2 = roc_auc_score(y, val_preds)
    
    print('v{} d{} f{} t{}: original ll {:.4f} auc {:.4f}, ensemble ll {:.4f} auc {:.4f}'
          .format(VERSION,ds_idx,fold,target,ll_val,auc_val,ll_val2,auc_val2))
    
    run_time = time.time() - st
    print('running time', run_time)
    
    stats = pd.DataFrame([[VERSION,ds_idx,fold,target,
                           ll_train,auc_train,ll_train2,auc_train2,
                           ll_val,auc_val,ll_val2,auc_val2,run_time]], 
                           columns = 
                            ['version','ds_idx','fold','target',
                             'train_loss','train_auc','train_loss_ens','train_auc_ens', 
                             'valid_loss','valid_auc','valid_loss_ens','valid_auc_ens',
                             'run_time'
                             ])

    stats_filename = PATH_DISK/'ensemble'/'stats.v{}'.format(VERSION)
    if stats_filename.is_file():
        stats = pd.concat([pd.read_csv(stats_filename), stats], sort=False)
    stats.to_csv(stats_filename, index=False)

#model.cols = Xt.columns
#predictions[data_filt['fold'] == i] = (Xv*model.x).sum(1)